### Baseline model for our project.

In [1]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

In [2]:
df = pd.read_csv('IMDB.csv')
df = df.sample(500)
df.to_csv('data.csv', index=False)
df.head()

,review,sentiment
161,not a Larry Mcmurty masterpiece but it stands ...,positive
932,"This should be re-named ""Everybody Loves Sebas...",negative
979,This is the kind of movie the US doesn't make....,positive
913,For those who have enjoyed the Asterix books a...,positive
138,Footlight Parade is among the best of the 1930...,positive


In [3]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

<>:32: SyntaxWarning: invalid escape sequence '\s'
<>:32: SyntaxWarning: invalid escape sequence '\s'
C:\Users\shaur\AppData\Local\Temp\ipykernel_6792\3798096103.py:32: SyntaxWarning: invalid escape sequence '\s'
  text = re.sub('\s+', ' ', text).strip()


In [4]:
df = normalize_text(df)
df.head()

,review,sentiment
161,larry mcmurty masterpiece stand good western l...,positive
932,re named everybody love sebastian rural go now...,negative
979,kind movie u make people rent foreign film gre...,positive
913,enjoyed asterix book film love film yes admit ...,positive
138,footlight parade among best s musical comedy e...,positive


In [5]:
df['sentiment'].value_counts()

sentiment
negative    255
positive    245
Name: count, dtype: int64

In [6]:
x = df['sentiment'].isin(['positive','negative'])
df = df[x]

In [7]:
df['sentiment'] = df['sentiment'].map({'positive':1, 'negative':0})
df.head()

,review,sentiment
161,larry mcmurty masterpiece stand good western l...,1
932,re named everybody love sebastian rural go now...,0
979,kind movie u make people rent foreign film gre...,1
913,enjoyed asterix book film love film yes admit ...,1
138,footlight parade among best s musical comedy e...,1


In [8]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [9]:
vectorizer = CountVectorizer(max_features=100)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [11]:
import dagshub

mlflow.set_tracking_uri('https://dagshub.com/shauryaraj5/sentiment_prediction_e2e.mlflow')
dagshub.init(repo_owner='shauryaraj5', repo_name='sentiment_prediction_e2e', mlflow=True)

# mlflow.set_experiment("Logistic Regression Baseline")
mlflow.set_experiment("Logistic Regression Baseline")


Accessing as shauryaraj5

Initialized MLflow to track repo "shauryaraj5/sentiment_prediction_e2e"

Repository shauryaraj5/sentiment_prediction_e2e initialized!

2025/10/12 20:08:06 INFO mlflow.tracking.fluent: Experiment with name 'Logistic Regression Baseline' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/a36b17b8209c4d0e85b628b2d2d1790b', creation_time=1760279887328, experiment_id='0', last_update_time=1760279887328, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [ ]:
import mlflow
import logging
import os
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 100)
        mlflow.log_param("test_size", 0.25)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)  # Increase max_iter to prevent non-convergence issues

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        logging.info("Saving and logging the model...")
        # mlflow.sklearn.log_model(model, "model")              Mlflow log model does not work in dagshub so intead saving it locally
        mlflow.sklearn.save_model(model, "exp_models/exp-1")

        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        # Save and log the notebook
        # notebook_path = "exp1_baseline_model.ipynb"
        # logging.info("Executing Jupyter Notebook. This may take a while...")
        # os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
        # mlflow.log_artifact(notebook_path)

        # logging.info("Notebook execution and logging complete.")

        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)


2025-10-12 20:08:10,341 - INFO - Starting MLflow run...
2025-10-12 20:08:11,860 - INFO - Logging preprocessing parameters...
2025-10-12 20:08:12,869 - INFO - Initializing Logistic Regression model...
2025-10-12 20:08:12,870 - INFO - Fitting the model...
2025-10-12 20:08:12,903 - INFO - Model training complete.
2025-10-12 20:08:12,904 - INFO - Logging model parameters...
2025-10-12 20:08:13,229 - INFO - Making predictions...
2025-10-12 20:08:13,231 - INFO - Calculating evaluation metrics...
2025-10-12 20:08:13,240 - INFO - Logging evaluation metrics...
2025-10-12 20:08:17,811 - INFO - Saving and logging the model...
2025/10/12 20:08:24 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025-10-12 20:08:24,595 - INFO - Model training and logging completed in 12.74 seconds.
2025-10-12 20:08:24,596 - INFO - Accuracy: 0.68
2025-10-12 20:08:24,597 - INFO - Precision: 0.7796610169491526
2

🏃 View run calm-worm-907 at: https://dagshub.com/shauryaraj5/sentiment_prediction_e2e.mlflow/#/experiments/0/runs/35a0684518064df4a1b95bdc30191dd7
🧪 View experiment at: https://dagshub.com/shauryaraj5/sentiment_prediction_e2e.mlflow/#/experiments/0
